In [6]:
exec(open("./variables.py").read())

In [7]:
cards = json.loads(json.dumps(requests.get(url_cards).json()))
lists = json.loads(json.dumps(requests.get(url_lists).json()))
customfields = json.loads(json.dumps(requests.get(url_customfields).json()))
labels = json.loads(json.dumps(requests.get(url_labels).json()))
members = json.loads(json.dumps(requests.get(url_members).json()))

In [8]:
customfields_dict = {}
for i in customfields:
    customfields_dict[i['id']] = {}
    if i['type'] == 'date':
        customfields_dict[i['id']][i['name']] = {}
        customfields_dict[i['id']][i['name']]['options'] = {'id': 'date'}
       

    else:
        customfields_dict[i['id']][i['name']] = {}
        customfields_dict[i['id']][i['name']]['options'] = {}
        for j in i['options']:
            customfields_dict[i['id']][i['name']]['options'][j['id']] =  j['value']['text']

customfieldsmetdate = []
for i,j in customfields_dict.items():
        for k,l in j.items():
            try:
                if l['options']['id'] == 'date':
                    customfieldsmetdate.append(i)
            except:
                pass

In [13]:
kaarten = {}
for i in cards:
    kaarten[i['id']] = {'name': i['name'],
                        'id': i['id'],
                        'idlist': i['idList'],
                        'customfields': i['customFieldItems'],
                        'labels': {},
                        'members': {},
                        'sjabloon': i['isTemplate'],
                        'due': None,
                        'closed': i['closed'],
                        'attachments': {}
                       }
    for j in i['idMembers']:

        for k in members:

            if j == k['id']:
                    kaarten[i['id']]['members'][k['id']] = k['fullName']
    if i['due'] != None:
        kaarten[i['id']]['due'] = datetime.strptime(i['due'][0:19],'%Y-%m-%dT%H:%M:%S')
    for j in i['labels']:
        kaarten[i['id']]['labels'][j['name']] = j['id']
    for j in i['attachments']:
        try:
            if j['url'][0:21]== 'https://trello.com/c/':
                kaarten[i['id']]['attachments'][j['url'][21:29]] = None
        except:
            pass
                    
if customfields_dict != {}:
    for i,j in customfields_dict.items():
        for k,l in j.items():
            for m,n in kaarten.items():
                n[k] = None

    for i,j in kaarten.items():
        for k in j['customfields']:
            if k['idCustomField'] in customfieldsmetdate:
                for l,m in customfields_dict.items():
                    for n,o in m.items():
                        if k['idCustomField'] == l:
                            j[n] = datetime.strptime(k['value']['date'][0:19],'%Y-%m-%dT%H:%M:%S')
            else:
                for l,m in customfields_dict.items():
                    for n,o in m.items():
                        if k['idCustomField'] == l:
                            for p,q in o.items():
                                for r,s in q.items():
                                    if k['idValue'] == r:
                                        j[n] = s     

for i,j in kaarten.items():
    date = idtodate(i)
    j['created'] = date
    for k in lists:
        if j['idlist'] == k['id']: j['list'] = k['name'] 
    if j['list'] in lijstenbeginnen:
        j['status'] = 'Niet gestart'
    elif j['list'] in lijstendoing:
        j['status'] = 'Doing'
    elif j['list'] in lijstenblocked:
        j['status'] = 'Blocked'
    elif j['list'] in lijstendone:
        j['status'] = 'Done'
    del j['customfields']
    del j['idlist']
    del j['id']



tedeletenlijsten = []

for i in lists:
    if i['name'] not in lijstenforscrum:
        tedeletenlijsten.append(i['name'])
tedeletenkaarten = []
for i,j in kaarten.items():
    if j['sjabloon'] == True:
        tedeletenkaarten.append(i)
    elif j['list'] in tedeletenlijsten:
        tedeletenkaarten.append(i)

for i in tedeletenkaarten:
    if i in kaarten:
        del kaarten[i]


In [14]:
df = pd.DataFrame(data=kaarten).T
df

,name,labels,members,sjabloon,due,closed,attachments,created,list,status
5c93850508325f533347135b,Digitaal werken USD,{},{},False,NaT,False,{},2019-03-21 12:35:17,Nieuw,Niet gestart
5c9387e22dcfdf15586fee7c,Werkinstructies USD breed,{},{},False,NaT,False,{},2019-03-21 12:47:30,Nieuw,Niet gestart
5c9390cd83e2aa1362ef1e08,Kwaliteit verbeteren,{},{},False,NaT,False,{},2019-03-21 13:25:33,Nieuw,Niet gestart
5dcd4e6dbf3b0b759de264ee,Communicatie en promotie USD,{},{},False,NaT,False,{},2019-11-14 12:54:05,Nieuw,Niet gestart
5df74c0047953868363c8c35,(MO) Met maatschappelijke partners komen tot e...,{'Teamplannen MO': '5df781647f1f092ec5f63b2c'},{},False,NaT,False,"{'Ys6UxKpc': None, 'VYWwPW5Y': None, 'mVMAPxvi...",2019-12-16 09:18:56,Nieuw,Niet gestart
...,...,...,...,...,...,...,...,...,...,...
5c6e69647866a53659cbc5f7,Er vinden tussentijdse evaluaties plaats bij i...,{},"{'593a5dac4fb7ccd1ecff7807': 'C Maandag', '5c8...",False,2019-05-31 10:00:00,False,{'TAJMHtp2': None},2019-02-21 09:03:32,Done,Done
5d88cdc536a5da732266f179,"Wmo: aanpassen verordening, beleidsregels en b...",{},{'58ef69408bb8da5f8b9199f2': 'Anita Muller'},False,NaT,False,{},2019-09-23 13:51:01,Done,Done
5c6e6955cfc092827830934a,Passend organiseren verder ontwikkelen,{},{'5a79afd3b6ff115227d33e05': 'Maarten Kragting'},False,2019-10-01 10:00:00,False,{},2019-02-21 09:03:17,Done,Done
5d97445d0e828c8ff8ed8915,NopNet,"{'Belangrijk': '5c6e7843b8d2620cba04b598', 'A3...",{'5c6fae780ee1393daac8e1ce': 'Alwin'},False,NaT,False,"{'xVJNKrpf': None, 'XgY1fZJq': None}",2019-10-04 13:08:45,Done,Done
